In [131]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from subprocess import check_output


# Pipeline at a glance:
load data -> cleaning/preprocessing -> feature eng -> format data for modelling -> fit model -> evaluate model -> generate submission file

## Step 1: Load data

In [132]:
df_train = pd.read_csv('../input/train.csv')
df_train.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [133]:
df_train.isnull().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

In [134]:
df_test = pd.read_csv('../input/test.csv')
df_test_duplicate = df_test

In [135]:
df_test.isnull().sum()

PassengerId      0
Pclass           0
Name             0
Sex              0
Age             86
SibSp            0
Parch            0
Ticket           0
Fare             1
Cabin          327
Embarked         0
dtype: int64

## Step 2: Basic cleaning and preprocessing

In [136]:
# label encode sex
le = LabelEncoder()
le.fit(df_train.Sex)
df_train.loc[:,'Sex'] = le.transform(df_train.Sex)
df_test.loc[:,'Sex'] = le.transform(df_test.Sex)

In [137]:
# Fill in the missing values
mAge = pd.concat((df_train.Age, df_test.Age), axis=0).mean()
df_train.loc[:, 'Age'] = df_train.Age.fillna(mAge)
df_test.loc[:, 'Age'] = df_test.Age.fillna(mAge)

## Step 3: Feature engineering

In [138]:
df_train = df_train.replace(np.nan, 'S', regex=True)
df_test = df_test.replace(np.nan, 'S', regex=True)

In [139]:
df_train['age_binned'] = pd.cut(df_train['Age'], [0, 16, 32, 48, 64, 200], labels=[0,1,2,3,4], retbins=False)
df_test['age_binned'] = pd.cut(df_test['Age'], [0, 16, 32, 48, 64, 200], labels=[0,1,2,3,4], retbins=False)

df_train['family_size'] = df_train['SibSp'] + df_train['Parch']
df_test['family_size'] = df_test['SibSp'] + df_test['Parch']

df_train['is_alone'] = np.where(df_train['family_size']==0, 1, 0)
df_test['is_alone'] = np.where(df_test['family_size']==0, 1, 0)

le.fit(df_train.Embarked)
df_train.loc[:,'Embarked'] = le.transform(df_train.Embarked)
df_test.loc[:,'Embarked'] = le.transform(df_test.Embarked)

df_train = df_train.drop(['PassengerId', 'Name', 'Age', 'SibSp', 'Parch', 'Ticket', 'Fare', 'Cabin'], axis=1)
df_test = df_test.drop(['PassengerId', 'Name', 'Age', 'SibSp', 'Parch', 'Ticket', 'Fare', 'Cabin'], axis=1)

## Step 4: Format data for modelling

In [140]:
# Split data into X and y, and select features to use
X = df_train.drop(['Survived'], axis=1)
y = df_train.loc[:,'Survived']

enc = OneHotEncoder(categorical_features = [0,1,2,3,5])
X = enc.fit_transform(X).toarray()

In [141]:
"""
# Split data X and y into train and val sets
#X_test = df_test.loc[:,['Pclass', 'Sex', 'Embarked', 'age_binned', 'family_size', 'is_alone']]
#X_test = df_train.loc[:,['Pclass', 'Sex', 'Embarked', 'age_binned', 'family_size', 'is_alone']]
X_test = df_train.drop(['Survived'], axis=1)
X_test = enc.fit_transform(X_test).toarray()
X_train, X_valid, y_train, y_valid = train_test_split(
      X, y, test_size=0.2, random_state=42)
"""
# Split data X and y into train and val sets
X_test = df_test.loc[:,['Pclass', 'Sex', 'Embarked', 'age_binned', 'family_size', 'is_alone']]
X_train, X_valid, y_train, y_valid = train_test_split(
      X, y, test_size=0.2, random_state=42)

## Step 5: Train (or fit) the model

In [142]:
# Fit logistic regression
RFC = RandomForestClassifier(n_estimators=320, max_depth=9)
RFC.fit(X_train, y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=9, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=320, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

## Step 6: Evaluate the model

In [143]:
# define a scoring function
def acc(y: np.array, y_pred: np.array) -> float:
    return np.sum(y_pred==y)/len(y)

In [144]:
# Use model to predict validation set
y_pred_valid = RFC.predict(X_valid)

In [145]:
print('Validation set accuracy', acc(y_valid, y_pred_valid), '%')

Validation set accuracy 0.8212290502793296 %


## Step 7: Generate Submission

In [146]:
X_test = enc.fit_transform(X_test).toarray()
X_test.shape

(418, 16)

In [147]:
# Predict for test set

y_pred_test = RFC.predict(X_test)

# Create a Kaggle submission
sub = pd.DataFrame({'PassengerId': df_test_duplicate['PassengerId'],
                    'Survived': y_pred_test})

sub.to_csv('kae_submission_RFC.csv', index=False)

In [96]:
'''
c = RandomForestClassifier()
p = {
    'n_estimators'      : [320,330,340],
    'max_depth'         : [8, 9, 10, 11, 12],
    'random_state'      : [0],
    'max_features': ['auto', 'sqrt', 'log2'],
    #'criterion' :['gini']
}
grid_search = GridSearchCV(estimator = c,
                           param_grid = p,
                           scoring = 'accuracy',
                           cv = 10,
                           verbose=1)
grid_search = grid_search.fit(X_train, y_train)
best_parameters = grid_search.best_params_
best_accuracy = grid_search.best_score_

'''

"\nc = RandomForestClassifier()\np = {\n    'n_estimators'      : [320,330,340],\n    'max_depth'         : [8, 9, 10, 11, 12],\n    'random_state'      : [0],\n    'max_features': ['auto', 'sqrt', 'log2'],\n    #'criterion' :['gini']\n}\ngrid_search = GridSearchCV(estimator = c,\n                           param_grid = p,\n                           scoring = 'accuracy',\n                           cv = 10,\n                           verbose=1)\ngrid_search = grid_search.fit(X_train, y_train)\nbest_parameters = grid_search.best_params_\nbest_accuracy = grid_search.best_score_\n\n"